# Семинар 8: ООП

Объектно ориентированное программирование на самом деле это по факту 4 принципа:
* **Abstraction** -- создавания классы мы меняем уровень абстракции, моделируем поведение класса и логику взаимодестсвия сущностей.
* **Наследование** -- нет смысла дублировать код, если можно выделить его в общего родителя для классов. Таким образом мы создаем новые абстракции на основе существующих.
* **Encapsulation** -- инкапсуляция, иными словами, скрытие внутри класса того, что не нужно явно использовать при работе с ним. Оставляем только публичный интерфейс работы с ним.
* **Polymorphism** -- разные классы могут реализовать разную логику одного и того же интерфейса.

Давайте напишем, например, свой класс комплексных чисел:

In [ ]:
from typing import Union


class Complex:
    def __init__(self, re: Union[int, float] = 0, im: Union[int, float] = 0):
        self.re = re
        self.im = im
        
    def __str__(self) -> str:
        if not self.re and not self.im:
            return "0"

        str_re = str(self.re) if self.re else ""
        str_im = str(abs(self.im)) if abs(self.im) not in (0, 1) else ""
        i_sign = "i" if self.im else ""

        if self.im < 0:
            operator = "-"
        elif self.im > 0 and self.re:
            operator = "+"
        else:
            operator = ""

        return str_re + operator + str_im + i_sign

Теперь добавим возможность складывать наши числа и умножать:

In [ ]:
class Complex:
    def __init__(self, re: Union[int, float] = 0, im: Union[int, float] = 0):
        self.re = re
        self.im = im
        
    def __str__(self) -> str:
        if not self.re and not self.im:
            return "0"

        str_re = str(self.re) if self.re else ""
        str_im = str(abs(self.im)) if abs(self.im) not in (0, 1) else ""
        i_sign = "i" if self.im else ""

        if self.im < 0:
            operator = "-"
        elif self.im > 0 and self.re:
            operator = "+"
        else:
            operator = ""

        return str_re + operator + str_im + i_sign
    
    __repr__ = __str__  # можно и свой repr написать, а можно и так

    def __add__(self, other: "Complex") -> "Complex":  # <--- обратите внимание на кавычки
        # с Python 3.11 можно будет вместо "Complex" писать просто Self
        return Complex(self.re + other.re, self.im + other.im)

    def __eq__(self, other: "Complex") -> "Complex":
        return self.re == other.re and self.im == other.im

    def __mul__(self, other: "Complex") -> "Complex":  # <--- обратите внимание на кавычки
        # с Python 3.11 можно будет вместо "Complex" писать просто Self
        return Complex(
            self.re * other.re - self.im * other.im,
            self.re * other.im + self.im * other.re
        )

    def __eq__(self, other: "Complex") -> bool:
        return self.re == other.re and self.im == other.im

    __radd__ = __add__  # автоматически подключим +=
    __rmul__ = __mul__  # автоматически подключим +=

    # аналогично можно использовать __sub__ для вычитания, __div__ для деления

### Наследование

По факту, наследованием мы расширяем наш класс, создавания от него потомка, например:

In [ ]:
import math

class Point(Complex):
    def length(self):
        return math.sqrt(self.re ** 2 + self.im ** 2)

In [ ]:
x = Point(5, 6)
y = Point(-1, 1)

print(x.length())
print(y.length())

print((x + y).length())  # а вот это выполнится? если нет, то как поправить?

На самом деле, можно было не писать тут свой метод length, а воспользоваться `__len__`

In [ ]:
import math

class Point(Complex):
    def __len__(self) -> float:
        return math.sqrt(self.re ** 2 + self.im ** 2)

    def distance(self, other: "Point") -> float:
        # YOUR CODE HERE
        pass

In [ ]:
x = Point(5, 6)
y = Point(-1, 1)

print(len(x))
print(len(y))

print(len(x + y))

### Задание 1

Немного "причешем" наше решение: давайте научим наш класс складываться и умножаться не только с комлексными числами, но и обычными

Для того, чтобы понять, какой это тип, можно использовать функцию `isinstance`.

### Задание 2

Давайте допишем класс Point, чтобы у него был метод distance, вычисляющий расстояние от нашей точки для другой. Как это лучше сделать?

### Ошибки

В питоне можно явно вызывать любую ошибку через raise, например:

In [ ]:
raise RuntimeError("Something went wrong")

А также можно создавать свои ошибки:

In [ ]:
class ComplexError(BaseException):
    ...

raise ComplexError("Invalid usage of complex numbers")

In [ ]:
class ComplexOperationError(BaseException):
    def __init__(self, left_arg, right_arg):
        self.left_arg = left_arg
        self.right_arg = right_arg

    def __str__(self) -> str:
        return f"Cannot do operation between {self.left_arg} and {self.right_arg}"

raise ComplexOperationError(Complex(1, 2), "abc")

### Задание 3

Допишите класс Complex таким образом, чтобы он возвращал ошибки, если мы пытаемся сложить/умножить, например, что-то невалидное (комплексное со строкой, например)